### Custom IAM Policy for Role, Policy Management and Elastic Container Registry

 Steps:
- Log in to the AWS Management Console using your Admin or Root User credentials by visiting the [AWS Console login page](https://aws.amazon.com/console/).
- Attach a Managed Policy IAMFullAccess and AmazonElasticContainerRegistryPublicFullAccess.
- Once you’ve attached the policy with sufficient permissions, switch back to your user and run the initial code to create roles, attach policies and create custom containers.

In [1]:
import os
import boto3
from RolePolicyManager import RolePolicyManager

%load_ext dotenv
%dotenv

In [2]:
account_id = os.environ["ACCOUNT_ID"]
domain_id = os.environ["DOMAIN_ID"]
region = os.environ["AWS_REGION"]
bucket = os.environ["BUCKET"]
user_name = os.environ["USER_NAME"]
role = os.environ["ROLE"]
role_name = os.environ["ROLE_NAME"]
policy_name = os.environ["POLICY_NAME"]

role_service = RolePolicyManager(account_id, user_name, role_name, policy_name, policy_name, bucket)
role_service.create_role_and_policy()

Successfully updated trust relationship for role: AmazonSageMaker-ExecutionRole-20240309T101533
Policy AmazonSageMaker-ExecutionPolicy-20240309T101533 already exists.
Policy arn: arn:aws:iam::284415450706:policy/AmazonSageMaker-ExecutionPolicy-20240309T101533.


In [3]:
from botocore.exceptions import ClientError
import boto3


def create_s3_bucket(s3_client, bucket_name, region=None):
    """
    Create an S3 bucket in a specified region. If no region is provided, the bucket will be created in the S3 default region (us-east-1).

    Args:
        bucket_name (str): The name of the bucket to create.
        region (str): The AWS region where the bucket will be created (optional).

    Returns:
        bool: True if bucket created successfully, False otherwise.
    """
    try:

        if region is None:
            s3_client.create_bucket(
                Bucket=bucket_name,
                CreateBucketConfiguration={
                    'LocationConstraint': 'eu-north-1'
                }
            )
        else:
            s3_client.create_bucket(
                Bucket=bucket_name,
                CreateBucketConfiguration={
                    'LocationConstraint': region
                }
            )

        print(f'Bucket "{bucket_name}" created successfully.')

    except ClientError as e:
        print(f'Error: {e}')

s3_client = role_service.create_client('s3')

create_s3_bucket(s3_client, bucket, region)

Error: An error occurred (BucketAlreadyOwnedByYou) when calling the CreateBucket operation: Your previous request to create the named bucket succeeded and you already own it.


In [4]:
from sagemaker.session import Session
from sagemaker.s3 import S3Uploader

s3_location = f"s3://{bucket}/football"
sagemaker_session = Session()

df_local_path = str(os.environ['DATA_FILEPATH_X'])
y_local_path = str(os.environ['DATA_FILEPATH_Y'])

s3_client.upload_file(Filename=df_local_path, Bucket=bucket, Key=f"data/df.csv")
s3_client.upload_file(Filename=y_local_path, Bucket=bucket, Key=f"data/y.csv")

sagemaker.config INFO - Not applying SDK defaults from location: C:\ProgramData\sagemaker\sagemaker\config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: C:\Users\kamil\AppData\Local\sagemaker\sagemaker\config.yaml


In [5]:
def create_ecr_repository(repository_name, ecr_client):
    try:
        response = ecr_client.describe_repositories(
            repositoryNames=[repository_name]
        )
        if response['repositories']:
            print(f"Repository already exists: {response['repositories'][0]['repositoryUri']}")
            return response['repositories'][0]
    except ClientError as e:
        if e.response['Error']['Code'] == 'RepositoryNotFoundException':
            try:
                response = ecr_client.create_repository(
                    repositoryName=repository_name,
                    tags=[
                        {
                            'Key': 'Project',
                            'Value': 'home-win-match-predictor'
                        }
                    ]
                )
                print(f"Repository created successfully: {response['repository']['repositoryUri']}")
                return response['repository']
            except ClientError as create_error:
                print(f"Error creating repository: {create_error}")
                return None
        else:
            print(f"Error describing repository: {e}")
            return None

In [6]:
processor_image_name = 'sagemaker-processing-container'
train_image_name = 'xgb-clf-training-container'

ecr_client = role_service.create_client('ecr')

create_ecr_repository(processor_image_name, ecr_client)
create_ecr_repository(train_image_name, ecr_client)

Repository already exists: 284415450706.dkr.ecr.eu-north-1.amazonaws.com/sagemaker-processing-container
Repository already exists: 284415450706.dkr.ecr.eu-north-1.amazonaws.com/xgb-clf-training-container


{'repositoryArn': 'arn:aws:ecr:eu-north-1:284415450706:repository/xgb-clf-training-container',
 'registryId': '284415450706',
 'repositoryName': 'xgb-clf-training-container',
 'repositoryUri': '284415450706.dkr.ecr.eu-north-1.amazonaws.com/xgb-clf-training-container',
 'createdAt': datetime.datetime(2024, 9, 10, 19, 49, 24, 466000, tzinfo=tzlocal()),
 'imageTagMutability': 'MUTABLE',
 'imageScanningConfiguration': {'scanOnPush': False},
 'encryptionConfiguration': {'encryptionType': 'AES256'}}

In [7]:
from LifecyclePolicyManager import LifecyclePolicyManager

lifecycle_policy = {
    "rules": [
        {
            "rulePriority": 1,
            "description": "Keep only one untagged image, expire all others",
            "selection": {
                "tagStatus": "any",
                "countType": "imageCountMoreThan",
                "countNumber": 1
            },
            "action": {
                "type": "expire"
            }
        }
    ]
}

lifecycle_manager = LifecyclePolicyManager(account_id)
lifecycle_manager.put_lifecycle_policy(ecr_client=ecr_client, image_name=processor_image_name, lifecycle_policy=lifecycle_policy)

Lifecycle policy attached to repository sagemaker-processing-container successfully.


In [ ]:
# from pythonProject

tag = ':latest'

train_image_uri = '{}.dkr.ecr.{}.amazonaws.com/{}'.format(account_id, region, train_image_name + tag)
print(f'Train image name: {train_image_uri}.')
!docker build -t $train_image_uri ../containers/training
!aws ecr get-login-password --region {region} | docker login --username AWS --password-stdin {account_id}.dkr.ecr.{region}.amazonaws.com
!docker push $train_image_uri

Train image name: 284415450706.dkr.ecr.eu-north-1.amazonaws.com/xgb-clf-training-container:latest.


#0 building with "default" instance using docker driver

#1 [internal] load build definition from Dockerfile
#1 transferring dockerfile: 381B done
#1 DONE 0.0s

#2 [auth] library/python:pull token for registry-1.docker.io
#2 DONE 0.0s

#3 [internal] load metadata for docker.io/library/python:3.10-slim
#3 DONE 1.0s

#4 [internal] load .dockerignore
#4 transferring context: 2B done
#4 DONE 0.0s

#5 [1/6] FROM docker.io/library/python:3.10-slim@sha256:80619a5316afae7045a3c13371b0ee670f39bac46ea1ed35081d2bf91d6c3dbd
#5 DONE 0.0s

#6 [internal] load build context
#6 transferring context: 65B done
#6 DONE 0.0s

#7 [2/6] RUN apt-get -y update && apt-get install -y --no-install-recommends     python3     build-essential     libssl-dev
#7 CACHED

#8 [4/6] RUN pip install --user --upgrade pip
#8 CACHED

#9 [5/6] RUN pip3 install -r requirements.txt
#9 CACHED

#10 [3/6] COPY requirements.txt .
#10 CACHED

#11 [6/6] COPY train.py /opt/ml/code/train.py
#11 CACHED

#12 exporting to image
#12 exporti

Login Succeeded


In [9]:
tag = ':latest'

processor_image_uri = '{}.dkr.ecr.{}.amazonaws.com/{}'.format(account_id, region, processor_image_name + tag)
print(f'Processor image name: {processor_image_uri}.')
# !docker build -t $processor_image_uri ../preprocessor/docker
# !aws ecr get-login-password --region {region} | docker login --username AWS --password-stdin {account_id}.dkr.ecr.{region}.amazonaws.com
# !docker push $processor_image_uri

Processor image name: 284415450706.dkr.ecr.eu-north-1.amazonaws.com/sagemaker-processing-container:latest.


In [10]:
import json
import subprocess

session_name = "FootballPredictorRoleSession"
image_directory = "../preprocessor/docker"
tag = ":latest"

os.environ['AWS_ACCESS_KEY_ID'] = credentials['AccessKeyId']
os.environ['AWS_SECRET_ACCESS_KEY'] = credentials['SecretAccessKey']
os.environ['AWS_SESSION_TOKEN'] = credentials['SessionToken']

# Define image parameters
processor_image_uri = f"{account_id}.dkr.ecr.{region}.amazonaws.com/{processor_image_name}{tag}"

# Step 3: Build Docker image
build_cmd = f"docker build -t {processor_image_uri} {image_directory}"
subprocess.run(build_cmd, shell=True, check=True)

# Step 4: Authenticate Docker to ECR
try:
    ecr_password = subprocess.run(
        ["aws", "ecr", "get-login-password", "--region", region],
        capture_output=True, text=True, check=True
    )
    print("ECR password retrieved successfully.")

    docker_login_cmd = f"echo {ecr_password.stdout} | docker login --username AWS --password-stdin {account_id}.dkr.ecr.{region}.amazonaws.com"
    subprocess.run(docker_login_cmd, shell=True, check=True)
    print("Docker login successful.")
except subprocess.CalledProcessError as e:
    print(f"Error occurred during ECR login: {e}")
    print(f"Stdout: {e.stdout}")
    print(f"Stderr: {e.stderr}")
    raise

# Step 4: Build and push Docker image
processor_image_uri = f"{account_id}.dkr.ecr.{region}.amazonaws.com/{processor_image_name}{tag}"
try:
    build_cmd = f"docker build -t {processor_image_uri} {image_directory}"
    subprocess.run(build_cmd, shell=True, check=True)
    print(f"Docker image {processor_image_uri} built successfully.")

    push_cmd = f"docker push {processor_image_uri}"
    subprocess.run(push_cmd, shell=True, check=True)
    print(f"Docker image {processor_image_uri} pushed successfully.")
except subprocess.CalledProcessError as e:
    print(f"Error occurred while building/pushing Docker image: {e}")
    raise




Error occurred during ECR login: Command '['aws', 'ecr', 'get-login-password', '--region', 'eu-north-1']' returned non-zero exit status 254.
Stdout: 
Stderr: 
An error occurred (AccessDeniedException) when calling the GetAuthorizationToken operation: User: arn:aws:sts::284415450706:assumed-role/AmazonSageMaker-ExecutionRole-20240309T101533/FootballPredictorRoleSession is not authorized to perform: ecr:GetAuthorizationToken on resource: * because no identity-based policy allows the ecr:GetAuthorizationToken action



CalledProcessError: Command '['aws', 'ecr', 'get-login-password', '--region', 'eu-north-1']' returned non-zero exit status 254.

In [ ]:
# Define variables
session_name = "FootballPredictorRoleSession"

# Assume role and get temporary credentials
t_credentials = f'$(aws sts assume-role --role-arn ${role} --role-session-name ${session_name} --region ${region} --output json)'

# Extract credentials from the JSON response
!export AWS_ACCESS_KEY_ID=$(echo $t_credentials | jq -r .Credentials.AccessKeyId)
!export AWS_SECRET_ACCESS_KEY=$(echo $t_credentials | jq -r .Credentials.SecretAccessKey)
!export AWS_SESSION_TOKEN=$(echo $t_credentials | jq -r .Credentials.SessionToken)

# Define image parameters
tag = ":latest"

# Construct the image URI
train_image_uri = f"${account_id}.dkr.ecr.${region}.amazonaws.com/${train_image_name}${tag}"
!echo f"Processor image URI: ${processor_image_uri}"

# Build the Docker image
!docker build -t ${processor_image_uri} ../preprocessor/docker

# Authenticate Docker to ECR
!aws ecr get-login-password --region ${region} | docker login --username AWS --password-stdin ${account_id}.dkr.ecr.${region}.amazonaws.com

# Push the Docker image to ECR
!docker push ${processor_image_uri}